In [15]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver


In [16]:
class ChatState(TypedDict):
    messages : Annotated[list[BaseMessage],add_messages]

In [17]:
llm = ChatOllama(model="llama3.2:latest")

In [18]:
def chat_node(state:ChatState) :
    # take user query from state
    messages = state['messages']
    #sending it to llm
    response = llm.invoke(messages)
    # response store into the state
    
    return {'messages': [response]}

In [20]:
checkpointer = MemorySaver()

graph = StateGraph(ChatState)

graph.add_node("chat_node", chat_node)

graph.add_edge(START,"chat_node")

graph.add_edge("chat_node", END)

#graph.compile()

workflow = graph.compile(checkpointer=checkpointer)

#print(workflow)

initial_state={
   'messages': HumanMessage(content="What is the capital of France?")
}

#print(workflow.invoke(initial_state)['messages'][-1].content)


In [ ]:
thread_id = '1'
while True:

    user_message = input("Type here :")

    print("User :" ,user_message)

    if user_message.strip().lower() in ['exit','bye','quit']:
        break

    config = {'configurable':{'thread_id':thread_id}}
    response = workflow.invoke({'messages':[HumanMessage(content=user_message)]},config=config)

    print("AI :",response['messages'][-1].content)



User : My name is Sharad
AI : Namaste Sharad. It's nice to meet you! Is there something I can help you with or would you like to chat?
User : what is my Name?
AI : Your name is Sharad, as we discussed earlier.
User : what is 10*10?
AI : Easy math question!

10 * 10 = 100
User : can you add 15 into the result
AI : Another calculation!

100 + 15 = 115
